<a href="https://colab.research.google.com/github/maurya-anuj/Eip4/blob/master/Assign5/Final_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/Deep Vision/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
%tensorflow_version 1.x

import cv2
# import json

import numpy as np
import pandas as pd

# from functools import partial
# from pathlib import Path 
# from tqdm import tqdm

from google.colab.patches import cv2_imshow

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder


# from keras.applications import VGG16
from keras.layers.core import Dropout
# from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
# from keras.optimizers import SGD
# from keras.preprocessing.image import ImageDataGenerator


#########################
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]


##########  AUGUMENTATION GEN.... CUTOUT  #####################

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    def __init__(self, df, batch_size=32, shuffle=True, augumentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augumentation = augumentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        image = image/255
        if self.augumentation is not None:
            image = self.augumentation.flow(image, shuffle=False).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=1)
print(train_df.shape, val_df.shape)

########################################################
pixel_level = False
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

########################################

from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
        # # set input mean to 0 over the dataset
        # featurewise_center=False,
        # # set each sample mean to 0
        # samplewise_center=False,
        # # divide inputs by std of dataset
        # featurewise_std_normalization=False,
        # # divide each input by its std
        # samplewise_std_normalization=False,
        # # apply ZCA whitening
        # zca_whitening=False,
        # # epsilon for ZCA whitening
        # zca_epsilon=1e-06,
        # # randomly rotate images in the range (deg 0 to 180)
        # rotation_range=10,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # # set range for random shear
        # shear_range=0.,
        # # set range for random zoom
        # zoom_range=0.1,
        # # set range for random channel shifts
        # channel_shift_range=0.,
        # # set mode for filling points outside the input boundaries
        # fill_mode='nearest',
        # # value used for fill_mode = "constant"
        # cval=0.,
        # # randomly flip images
        # horizontal_flip=True,
        # # randomly flip images
        # vertical_flip=False,
        # # set rescaling factor (applied before any other transformation)
        # rescale=1.0/255.0,
        # set function that will be applied on each input
        preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=pixel_level),
        # # image data format, either "channels_first" or "channels_last"
        # data_format=None,
        # # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, augumentation = aug)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False, augumentation = None)  # val_df

# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}

#########################################################################################
from keras.models import model_from_json
json_file = open("/content/gdrive/My Drive/Deep Vision/resnet34_new.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.layers.pop() # Remove last dense layer

neck = model.output

def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck

def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))

model = Model(
    inputs=model.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

####################  COMPILE  ############################
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
# Prepare callbacks for model saving and for learning rate adjustment.

# Prepare model model saving directory.
import os
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

model_save = keras.callbacks.ModelCheckpoint('model{epoch:08d}.h5', period=5) 

lr_reducer = ReduceLROnPlateau(factor=.05,
                               cooldown=0,
                               patience=5,
                               verbose=1,
                               min_delta=0.01,
                               min_lr=0.5e-6)
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 40:
        lr *= 0.5e-2
    elif epoch > 30:
        lr *= 1e-2
    elif epoch > 20:
        lr *= 0.5e-1
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [lr_scheduler, model_save]

from keras.optimizers import SGD
opt = SGD(lr=0.0001)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

######################  FIT  #######################################
model_info = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    callbacks = callbacks
)

(11537, 28) (2036, 28)
Epoch 1/50
Learning rate:  0.001
360/360 [==============================] - 187s 520ms/step - loss: 10.8877 - gender_output_loss: 0.6886 - image_quality_output_loss: 1.0758 - age_output_loss: 1.5842 - weight_output_loss: 1.3265 - bag_output_loss: 1.0680 - footwear_output_loss: 1.0905 - pose_output_loss: 1.0565 - emotion_output_loss: 1.2957 - gender_output_acc: 0.5625 - image_quality_output_acc: 0.5411 - age_output_acc: 0.3807 - weight_output_acc: 0.6096 - bag_output_acc: 0.5492 - footwear_output_acc: 0.3950 - pose_output_acc: 0.6011 - emotion_output_acc: 0.6771 - val_loss: 10.6581 - val_gender_output_loss: 0.6873 - val_image_quality_output_loss: 1.0553 - val_age_output_loss: 1.5689 - val_weight_output_loss: 1.2717 - val_bag_output_loss: 1.0429 - val_footwear_output_loss: 1.0822 - val_pose_output_loss: 1.0236 - val_emotion_output_loss: 1.2244 - val_gender_output_acc: 0.5635 - val_image_quality_output_acc: 0.5559 - val_age_output_acc: 0.3659 - val_weight_output_acc